In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.ndimage.filters import uniform_filter1d
from matplotlib.gridspec import GridSpec
import geopandas as gpd
import regionmask

from glob import glob
from cartopy.util import add_cyclic_point
from scipy.stats import ttest_rel

from scipy.stats import linregress
import statsmodels.api as sm

In [10]:
def get_cell_numbers(num): #returns how many grid cells are in ice shelf mask
    mask = shelf_mask_da.where((shelf_mask_da==num))  
    mask = mask*el_mask_da
        
    mask2 = mask.dropna(dim = 'lon', how='all')
    mask2 = mask2.dropna(dim = 'lat', how='all')
    non_nans = (~np.isnan(mask2.values)).sum()

    return non_nans

## For ice shelves: ##
def get_shelf_mask(area_num):

    mask = shelf_mask_da.where((shelf_mask_da==area_num))      
        
    mask2 = mask.dropna(dim = 'lon', how='all')
    mask2 = mask2.dropna(dim = 'lat', how='all')
    mask_lat = mask2.lat.values
    mask_lon = mask2.lon.values
    
    return mask, mask_lat, mask_lon

def clip_data_to_mask(ds, area_num):
    mask, mask_lat, mask_lon= get_shelf_mask(area_num)
    
    if run=='merra':
        mask_np = mask.sel(lat = slice(np.min(mask_lat), np.max(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon))).values
        low_el_mask = el_mask_da.sel(lat = slice(np.min(mask_lat), np.max(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon))).values
        area = area_da.sel(lat = slice(np.min(mask_lat), np.max(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon)))
        clipped_ds = ds.sel(lat = slice(np.min(mask_lat), np.max(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon)))
        
    else:
        mask_np = mask.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon))).values
        low_el_mask = el_mask_da.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon))).values
        area = area_da.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon)))
        clipped_ds = ds.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon)))
    

    mask_np[~np.isnan(mask_np)]=1
    shelf_area = (mask_np*area*low_el_mask).sum(dim = ['lat','lon'])

    ts = (clipped_ds*mask*area*low_el_mask).sum(dim = ['lat','lon'])/shelf_area
    #variance = np.sqrt(((clipped_ds - ts)**2*area*mask*low_el_mask).sum(dim = ['lat','lon'])/shelf_area)
    return ts

## For ice shelf regions: ##
def get_region_mask(area):

    if area == 'Ross Sea':
        mask = shelf_mask_da.where((shelf_mask_da==69)|(shelf_mask_da==70)|
                                  (shelf_mask_da==63)|(shelf_mask_da==64))
    elif area == 'Ross': #east and west ross ice shelf
        mask = shelf_mask_da.where((shelf_mask_da==69)|(shelf_mask_da==70))
    elif area == 'Ronne-Filchner':#ronne-filchner
        mask = shelf_mask_da.where((shelf_mask_da==33)|(shelf_mask_da==34))
    elif area == 'Peninsula':#larsen c & d, G6, Bach, wilkins
        mask = shelf_mask_da.where((shelf_mask_da==51)|(shelf_mask_da==44)|(shelf_mask_da==5)|
                                   (shelf_mask_da==7)|(shelf_mask_da==45)|(shelf_mask_da==49)|
                                   (shelf_mask_da==50))
    elif area == 'Amery Region':
        mask = shelf_mask_da.where((shelf_mask_da==114)|(shelf_mask_da==101))
    elif area == 'Wilkes Land': 
        mask = shelf_mask_da.where((shelf_mask_da==99)|(shelf_mask_da==23)|(shelf_mask_da==156)
                                  |(shelf_mask_da==97)|(shelf_mask_da==155))
    elif area == 'Amundsen Sea':
        mask = shelf_mask_da.where((shelf_mask_da==56)|(shelf_mask_da==57)|(shelf_mask_da==55)|(shelf_mask_da==59)|
                                   (shelf_mask_da==54)|(shelf_mask_da==109)|(shelf_mask_da==52)|(shelf_mask_da==53))

    elif area == 'Victoria Land':
        mask = shelf_mask_da.where((shelf_mask_da==90)|(shelf_mask_da==89)|(shelf_mask_da==82)|(shelf_mask_da==72))
    elif area == 'DML':
        mask = shelf_mask_da.where((shelf_mask_da==124)|(shelf_mask_da==35)|(shelf_mask_da==36)|(shelf_mask_da==38)
                                  |(shelf_mask_da==123)|(shelf_mask_da==157)|(shelf_mask_da==120)
                                  |(shelf_mask_da==118)|(shelf_mask_da==117)|(shelf_mask_da==37)
                                  |(shelf_mask_da==119)|(shelf_mask_da==121))
    elif area == 'all ice shelves':
        mask = shelf_mask_da.where((shelf_mask_da==69)|(shelf_mask_da==70)|(shelf_mask_da==33)|
                                   (shelf_mask_da==34)|(shelf_mask_da==51)|(shelf_mask_da==44)|
                                   (shelf_mask_da==5)|(shelf_mask_da==7)|(shelf_mask_da==45)|
                                   (shelf_mask_da==49)|(shelf_mask_da==50)|(shelf_mask_da==114)|
                                   (shelf_mask_da==99)|(shelf_mask_da==101)|(shelf_mask_da==23)|
                                   (shelf_mask_da==56)|(shelf_mask_da==57)|(shelf_mask_da==55)|
                                   (shelf_mask_da==59)|(shelf_mask_da==54)|(shelf_mask_da==109)|
                                   (shelf_mask_da==52)|(shelf_mask_da==53)|(shelf_mask_da==90)|
                                   (shelf_mask_da==97)|(shelf_mask_da==156)|(shelf_mask_da==89)|
                                   (shelf_mask_da==63)|(shelf_mask_da==64)|(shelf_mask_da==72)|
                                   (shelf_mask_da==124)|(shelf_mask_da==35)|(shelf_mask_da==36)|
                                   (shelf_mask_da==38)|(shelf_mask_da==123)|(shelf_mask_da==157)|
                                   (shelf_mask_da==120)|(shelf_mask_da==118)|(shelf_mask_da==117)|
                                   (shelf_mask_da==37)|(shelf_mask_da==119)|(shelf_mask_da==121)|
                                   (shelf_mask_da==82)|(shelf_mask_da==155))

        
    #mask = mask*el_mask_da.values
        
    mask2 = mask.dropna(dim = 'lon', how='all')
    mask2 = mask2.dropna(dim = 'lat', how='all')
    mask_lat = mask2.lat.values
    mask_lon = mask2.lon.values
    
    return mask, mask_lat, mask_lon

def clip_data_to_region(ds, region):
    mask, mask_lat, mask_lon= get_region_mask(region)

    if run=='merra':
        mask_np = mask.sel(lat = slice(np.min(mask_lat), np.max(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon))).values
        low_el_mask = el_mask_da.sel(lat = slice(np.min(mask_lat), np.max(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon))).values
        area = area_da.sel(lat = slice(np.min(mask_lat), np.max(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon)))
        clipped_ds = ds.sel(lat = slice(np.min(mask_lat), np.max(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon)))
        
    else:
        mask_np = mask.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon))).values
        low_el_mask = el_mask_da.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon))).values
        area = area_da.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon)))
        clipped_ds = ds.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon)))
    
    mask_np[~np.isnan(mask_np)]=1
    shelf_area = (mask_np*area*low_el_mask).sum(dim = ['lat','lon'])
    ts = (clipped_ds*mask*area*low_el_mask).sum(dim = ['lat','lon'])/shelf_area
    variance = np.sqrt(((clipped_ds - ts)**2*area*mask*low_el_mask).sum(dim = ['lat','lon'])/shelf_area)
    return ts


def create_pandas_row(hist_data, ssp1_data, ssp3_data, ssp5_data,i, ice_shelf = True):
    d = {}
    
    if ice_shelf:
        d['name'] = shelves.loc[i].NAME
        hist = clip_data_to_mask(hist_data, i)
        ssp1 = clip_data_to_mask(ssp1_data, i)
        ssp3 = clip_data_to_mask(ssp3_data, i)
        ssp5 = clip_data_to_mask(ssp5_data, i)
    else:
        d['name'] = i
        hist = clip_data_to_region(hist_data, i)
        ssp1 = clip_data_to_region(last_ssp1, i)
        ssp3 = clip_data_to_region(last_ssp3, i)
        ssp5 = clip_data_to_region(last_ssp5, i)
        
    d['hist'] = float(hist.median().values)
    d['ssp1'] = float(ssp1.median().values)
    d['ssp3'] = float(ssp3.median().values)
    d['ssp5'] = float(ssp5.median().values)

    return d

In [11]:
shelves = gpd.read_file("masks/ice_shelves.geojson")
shelves.at[69, 'NAME']='Ross west'
shelves.at[70, 'NAME']='Ross east'

## Individual ice shelves

In [16]:
run = 'TP3'
var = 'fac'
shelf_data =  pd.DataFrame(columns = ['name','hist', 'ssp1','ssp3','ssp5'])

ssp1_data = xr.open_dataset(f'netcdf_results/emulator_results/ssp1_fac_{run}.nc')
ssp3_data = xr.open_dataset(f'netcdf_results/emulator_results/ssp3_fac_{run}.nc')
ssp5_data = xr.open_dataset(f'netcdf_results/emulator_results/ssp5_fac_{run}.nc')

# if run == 'merra':
#     hist_data = xr.open_dataset(f'netcdf_results/deg0C/merra_hist_1985-2015.nc')
#     shelf_mask_da = xr.open_dataarray('masks/merra/shelf_mask.nc')
#     el_mask_da = xr.open_dataarray('masks/merra/low_elevation.nc')
#     area_da = xr.open_dataarray('masks/merra/area.nc')
# else:
hist_data = xr.open_dataset(f'netcdf_results/emulator_results/era_{run}.nc')
shelf_mask_da = xr.open_dataarray('masks/era/shelf_mask.nc')
el_mask_da = xr.open_dataarray('masks/era/low_elevation.nc')
area_da = xr.open_dataarray('masks/era/area.nc')

last_ssp5 = ssp5_data.isel(year = -1)[var]#.quantile(0.5, dim='model')
last_ssp3 = ssp3_data.isel(year = -1)[var]#.quantile(0.5, dim='model')
last_ssp1 = ssp1_data.isel(year = -1)[var]#.quantile(0.5, dim='model')
hist_data2 = hist_data[var]


for i in range(len(shelves)):
    n = get_cell_numbers(i)
    if n > 5:
        d = create_pandas_row(hist_data2, last_ssp1, last_ssp3, last_ssp5,i)
        shelf_data = shelf_data.append(d, ignore_index=True)
for r in ['Ross Sea', 'Ross', 'Peninsula', 'Ronne-Filchner', 'DML', 'Amery Region', 'Wilkes Land', 'Victoria Land', 'Amundsen Sea', 'all ice shelves']:
    d = create_pandas_row(hist_data2, last_ssp1, last_ssp3, last_ssp5,r, ice_shelf = False)
    shelf_data = shelf_data.append(d, ignore_index=True)
        
shelf_data.to_csv(f'csvs/{var}_{run}.csv')